---
# Mounting Google Drive locally


In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


---
# Hardware

In [105]:
print('Hardware specs used:\n')
!lscpu |grep 'Model name'
!nvidia-smi -L
!free -h --si | awk  '/Mem:/{print $2}'
!df -h / | awk '{print $4}'

Hardware specs used:

Model name:          Intel(R) Xeon(R) CPU @ 2.00GHz
GPU 0: Tesla P100-PCIE-16GB (UUID: GPU-ad3abf54-8df1-21b5-49f5-62ca8e7ef600)
13G
Avail
34G


---
# Importing of the required packages

In [104]:
import os
import cv2
import time
import yaml
import json
import argparse
import numpy as np
import seaborn as sns
import tensorflow as tf
import tensorflow_hub as hub
import matplotlib.pyplot as plt
from sklearn.preprocessing import MultiLabelBinarizer
from tensorflow.keras import models
print('Tensorflow version: {}'.format(tf.__version__))
os.chdir('/content/drive/My Drive/keypoint_tracking')
pwd_dir = !pwd
print('\nCurrent working directory: {}'.format(pwd_dir[0]))

Tensorflow version: 2.2.0-rc2

Current working directory: /content/drive/My Drive/keypoint_tracking


---
# Handler functions

**Function randomly choosing image for prediction**

In [0]:
def get_random_img(dir):
    img = os.path.join(dir, random.choice(os.listdir(dir)))
    while os.path.isdir(img):
        img = os.path.join(img, random.choice(os.listdir(img)))
    return img

**Function used for loading a model**

In [0]:
def load_model(model_dir):
    print('-' * 100)
    print('Loading the model and its weights...')
    start = time.time()
    architecture_path = os.path.join(model_dir, 'architecture.json')
    weights_path = os.path.join(model_dir, 'weights.h5')
    with open(architecture_path, 'r') as f:
        model = models.model_from_json(f.read(), custom_objects={'KerasLayer': hub.KerasLayer})
    model.load_weights(weights_path)
    end = time.time()
    print('Loading the model and its weights takes ({:1.3f} seconds)'.format(end - start))
    print('-' * 100)
    return model

>**Pre-processing an image for a model**

In [0]:
def process_images(paths, img_height, img_width, img_channels):
    img_inputs = np.ndarray(shape=(len(paths), img_height, img_width, img_channels), dtype=np.float32)
    idx = 0
    start = time.time()
    for path in paths:
        img_string = tf.io.read_file(path)
        img_input = tf.image.decode_png(img_string, channels=img_channels)
        img_resized = tf.image.resize(images=img_input, size=(img_height, img_width))
        img_norm = img_resized / 255.0
        img_inputs[idx] = img_norm
        idx += 1
    proc_time = time.time() - start
    print('-' * 100)
    print('Total pre-processing time......: {:1.3f} seconds'.format(proc_time))
    print('Average pre-processing time....: {:1.3f} seconds per image'.format(proc_time / img_inputs.shape[0]))
    print('Average pre-processing FPS.....: {:1.1f} frames per second'.format(1. / (proc_time / img_inputs.shape[0])))
    print('-' * 100)
    return img_inputs

>**Generating output predictions for the input samples**

In [0]:
def test_model(test_model_dir, test_files):
    # -------------------------------------- Getting of a YAML configuration ---------------------------------------
    config_path = os.path.join(test_model_dir, 'config.yaml')
    with open(config_path, 'r') as f:
        config = yaml.load(f)
    args.img_size = config['img_size']['value']
    args.model_name = config['model_name']['value']

    # -------------------------------------------- Show testing options --------------------------------------------
    print('-' * 100)
    print('Testing options:')
    print('Test model name...: {}'.format(args.model_name))
    print('Test model dir....: {}'.format(test_model_dir))
    print('Tested images.....: {}'.format(test_files))
    print('-' * 100)

    # --------------------------------- Model loading and getting of the prediction --------------------------------
    model = load_model(model_dir=test_model_dir)
    test_ds = process_images(paths=test_files,
                             img_height=args.img_size[0],
                             img_width=args.img_size[1],
                             img_channels=args.img_size[2])
    num_images = test_ds.shape[0]
    # test_ds = tf.data.Dataset.from_tensor_slices(test_files)
    # test_ds = test_ds.map(map_func=map_func, num_parallel_calls=tf.data.experimental.AUTOTUNE)
    # test_ds = test_ds.batch(batch_size=1)
    # num_images = len(test_files)
        
    # -------------------------- Generate prediction and process probabilities and label ---------------------------
    mlb = MultiLabelBinarizer(classes=args.label_names)
    mlb.fit(y=args.label_names)
    start = time.time()
    model_probs = model.predict(test_ds, verbose=0)
    pred_time = time.time() - start
    print('Total prediction time..........: {:1.3f} seconds'.format(pred_time))
    print('Average prediction time........: {:1.3f} seconds per image'.format(pred_time / num_images))
    print('Average prediction FPS.........: {:1.1f} frames per second'.format(1. / (pred_time / num_images)))
    print('-' * 100)
    return model_probs

>**Processing of the probabilities and labels** 

In [0]:
def process_predictions(model_output, test_files, thresh_label, thresh_x, thresh_y):
        batch_labels = []
        batch_label_probs = []
        batch_point_coords = []
        start = time.time()

        #data_processor = DataProcessor()
        if test_files[0].endswith('avi'):
            batch_images = data_processor.process_video(path=test_files,
                                                        img_height=1000,
                                                        img_width=1000,
                                                        img_channels=3)
        else:
            batch_images = process_images(paths=test_files,
                                          img_height=1000,
                                          img_width=1000,
                                          img_channels=3)
        num_files = batch_images.shape[0]
        img_size = [batch_images.shape[1], batch_images.shape[2], batch_images.shape[3]]
        for idx in range(num_files):
            inp_label_probs = model_output[0][idx]
            inp_label_probs = np.expand_dims(inp_label_probs, axis=0)
            inp_point_coords = model_output[1][idx]
            inp_point_coords = np.expand_dims(inp_point_coords, axis=0)

            if 2 * inp_label_probs.shape[1] != inp_point_coords.shape[1]:
                raise ValueError('Number of classes and coordinates must be equal!')

            # Get a list of the remaining points
            inp_label_bin = inp_label_probs > thresh_label
            x_coords = np.zeros((1, inp_label_probs.shape[1]), dtype=np.float)
            y_coords = np.zeros((1, inp_label_probs.shape[1]), dtype=np.float)
            for i in range(inp_label_probs.shape[1]):
                x_coords[0, i] = inp_point_coords[0, 2 * i]
                y_coords[0, i] = inp_point_coords[0, 2 * i + 1]
            x_coords_bin = x_coords > thresh_x
            y_coords_bin = y_coords > thresh_y
            inp_point_bin = np.logical_and(x_coords_bin, y_coords_bin)
            out_bin = np.logical_and(inp_label_bin, inp_point_bin)

            # Get classification labels
            mlb = MultiLabelBinarizer(classes=args.label_names)
            mlb.fit(y=args.label_names)
            out_labels = mlb.inverse_transform(yt=out_bin)
            out_labels = list(out_labels[0])

            # Get classification probabilities
            deleted_points = np.argwhere(out_bin == False)
            deleted_points = list(deleted_points[:, 1])
            out_label_probs = np.delete(inp_label_probs, deleted_points)
            out_label_probs = np.round(out_label_probs, decimals=2)
            out_label_probs = np.expand_dims(out_label_probs, axis=0)

            # Get regression coordinates
            x_coords *= img_size[0]
            y_coords *= img_size[1]
            x_coords = x_coords[out_bin]
            x_coords = np.expand_dims(x_coords, axis=0)
            y_coords = y_coords[out_bin]
            y_coords = np.expand_dims(y_coords, axis=0)
            out_point_coords = np.concatenate((x_coords, y_coords))
            out_point_coords = np.round(out_point_coords, decimals=0)
            out_point_coords = out_point_coords.astype(int)

            batch_labels.append(out_labels)
            batch_label_probs.append(out_label_probs)
            batch_point_coords.append(out_point_coords)
        proc_time = time.time() - start
        print('Total post-processing time.....: {:1.3f} seconds'.format(proc_time))
        print('Average post-processing time...: {:1.3f} seconds per image'.format(proc_time / len(test_files)))
        print('Average post-processing FPS....: {:1.1f} frames per second'.format(1. / (proc_time / len(test_files))))
        print('-' * 100)
        return batch_images, batch_labels, batch_label_probs, batch_point_coords

> **Showing images with points drawn**

In [0]:
def show_predictions(images, labels, probs, coords, verbose, shape, add_label, add_prob, palette):
    for idx, (image, label, prob, coord) in enumerate(zip(images, labels, probs, coords)):
        img_path = args.test_files[idx]
        if verbose == 0:
            print('-' * 100)
            print('Image...........: {}'.format(img_path))
            print('Labels..........: {}'.format(label))
            print('Probabilities...: {}'.format(label))
            print('X coordinates...: {}'.format(list(coord[0])))
            print('Y coordinates...: {}'.format(list(coord[1])))
            print('-' * 100)
        elif verbose == 1:
            img_labeled = put_points_on_image(image, label, prob, coord,
                                              shape=shape, add_label=add_label,
                                              add_prob=add_prob, palette=palette)
            plt.figure(img_path, figsize=(9, 12))
            plt.imshow(img_labeled, cmap='gray')
            title = '\n\nLabels: {}\n\nProbabilities: {}\n\nX coordinates: {}\n\nY coordinates: {}'\
                .format(label, list(prob[0]), list(coord[0]), list(coord[1]))
            plt.title(title, fontsize=14)
            plt.show()
        else:
            raise ValueError('Incorrect VERBOSE value, please check it!')

> **Saving images with points drawn**

In [0]:
def save_predictions(images, labels, probs, coords, save_dir, shape, add_label, add_prob, palette):
    model_dir = os.path.basename(args.test_model_dir)
    save_dir = os.path.join(save_dir, model_dir)
    if not os.path.isdir(save_dir):
        os.makedirs(save_dir)
    for idx, (image, label, prob, coord) in enumerate(zip(images, labels, probs, coords)):
        img_path = args.test_files[idx]
        pred_path = img_path[:-4] + '_' + os.path.split(args.test_model_dir)[1] + '.png'
        img_labeled = put_points_on_image(image, label, prob, coord,
                                          shape=shape, add_label=add_label,
                                          add_prob=add_prob, palette=palette)
        img_labeled_uint8 = (img_labeled*255).astype(np.uint8)
        img_bgr = cv2.cvtColor(img_labeled_uint8, cv2.COLOR_RGB2BGR)
        fname = os.path.join(save_dir, os.path.basename(pred_path))
        print('Image {} saved to {}'.format(os.path.split(img_path)[1], fname))
        cv2.imwrite(fname, img_bgr)

> **Function drawing points on an image**

In [0]:
def put_points_on_image(img, label, prob, coord, shape, add_label, add_prob, palette):
    img_src = img.copy()
    scale = 0.025
    fontScale = min(img_src.shape[0], img_src.shape[1]) / (25 / scale)
    palette = sns.color_palette(palette, n_colors=11)
    point_colors = {'AA1': palette[0], 'AA2':  palette[1], 'STJ1':  palette[2], 'STJ2':  palette[3],
                    'CD':  palette[4], 'CM':  palette[5], 'CP':  palette[6], 'CT':  palette[7], 'PT':  palette[8],
                    'FE1':  palette[9], 'FE2':  palette[10]}
    for idx, point_label in enumerate(label):
        point_prob = prob[0, idx]
        center_coord = (coord[0, idx], coord[1, idx])
        point_color = point_colors[point_label]
        if shape == 'circle':
            cv2.circle(img=img_src, center=center_coord, radius=7, color=point_color, thickness=-1)
        elif shape == 'star':
            radius = 7
            line_thick = 2
            alpha = 45 * np.pi/180
            c_x = coord[0, idx]
            c_y = coord[1, idx]
            p0_x = int(np.round(c_x))
            p0_y = int(np.round(c_y - radius))
            p1_x = int(np.round(c_x + np.cos(alpha)*radius))
            p1_y = int(np.round(c_y - np.sin(alpha)*radius))
            p2_x = int(np.round(c_x + radius))
            p2_y = int(np.round(c_y))
            p3_x = int(np.round(c_x + np.cos(alpha)*radius))
            p3_y = int(np.round(c_y + np.sin(alpha)*radius))
            p4_x = int(np.round(c_x))
            p4_y = int(np.round(c_y + radius))
            p5_x = int(np.round(c_x - np.cos(alpha)*radius))
            p5_y = int(np.round(c_y + np.sin(alpha)*radius))
            p6_x = int(np.round(c_x - radius))
            p6_y = int(np.round(c_y))
            p7_x = int(np.round(c_x - np.cos(alpha)*radius))
            p7_y = int(np.round(c_y - np.sin(alpha)*radius))
            cv2.line(img=img_src, pt1=(p0_x, p0_y), pt2=(p4_x, p4_y), color=point_color, thickness=line_thick, lineType=cv2.LINE_AA)
            cv2.line(img=img_src, pt1=(p1_x, p1_y), pt2=(p5_x, p5_y), color=point_color, thickness=line_thick, lineType=cv2.LINE_AA)
            cv2.line(img=img_src, pt1=(p2_x, p2_y), pt2=(p6_x, p6_y), color=point_color, thickness=line_thick, lineType=cv2.LINE_AA)
            cv2.line(img=img_src, pt1=(p3_x, p3_y), pt2=(p7_x, p7_y), color=point_color, thickness=line_thick, lineType=cv2.LINE_AA)
        elif shape == 'square':
            side = 7
            start_point = (coord[0, idx] - side, coord[1, idx] - side)
            end_point = (coord[0, idx] + side, coord[1, idx] + side)
            cv2.rectangle(img=img_src, pt1=start_point, pt2=end_point, color=point_color, thickness=-1)
        else:
            raise ValueError('Incorrect shape value, please check it!')

        if add_label and not add_prob:
            text = point_label
            text_coord = (center_coord[0] + 10, center_coord[1])
            cv2.putText(img=img_src, text=text, org=text_coord, color=point_color,
                        fontFace=cv2.FONT_HERSHEY_DUPLEX, fontScale=fontScale, thickness=1, lineType=cv2.LINE_AA)
        elif add_prob and not add_label:
            text = str(np.round(100*point_prob).astype(int))
            text_coord = (center_coord[0] + 10, center_coord[1])
            cv2.putText(img=img_src, text=text, org=text_coord, color=point_color,
                        fontFace=cv2.FONT_HERSHEY_DUPLEX, fontScale=fontScale, thickness=1, lineType=cv2.LINE_AA)
        elif add_prob and add_label:
            text = point_label + '(' + str(np.round(100*point_prob).astype(int)) + ')'
            text_coord = (center_coord[0] + 10, center_coord[1])
            cv2.putText(img=img_src, text=text, org=text_coord, color=point_color,
                        fontFace=cv2.FONT_HERSHEY_DUPLEX, fontScale=fontScale, thickness=1, lineType=cv2.LINE_AA)
    # Debugging only
    # plt.imshow(img_src)
    return img_src

---
# Initial parameters for model testing


Available models: MobileNet_V2, MobileNet_V2_ft, ResNet_V2, ResNet_V2_ft, Inception_V3, Inception_V3_ft 

In [0]:
TEST_MODEL_DIR = 'models/ResNet_V2'
TEST_FILES = ['data/img/001_025.png', 'data/img/002_028.png', 'data/img/003_032.png', 'data/img/004_016.png']
LABEL_NAMES = ['AA1', 'AA2', 'STJ1', 'STJ2', 'CD', 'CM', 'CP', 'CT', 'PT', 'FE1', 'FE2']
POINT_NAMES = ['AA1_x', 'AA1_y', 'AA2_x', 'AA2_y', 'STJ1_x', 'STJ1_y', 'STJ2_x', 'STJ2_y', 'CD_x', 'CD_y',
               'CM_x', 'CM_y', 'CP_x', 'CP_y', 'CT_x', 'CT_y', 'PT_x', 'PT_y', 'FE1_x', 'FE1_y', 'FE2_x', 'FE2_y']
parser = argparse.ArgumentParser(description='Keypoint tracking and classification')
# Testing arguments
parser.add_argument('-tmd', '--test_model_dir', metavar='', default=TEST_MODEL_DIR, type=str, help='model directory for testing mode')
parser.add_argument('-tf', '--test_files', nargs='+', metavar='', default=TEST_FILES, type=str, help='list of tested images')
# Common arguments
parser.add_argument('--label_names', metavar='', default=LABEL_NAMES, type=list, help='list of label names')
parser.add_argument('--point_names', metavar='', default=POINT_NAMES, type=list, help='list of point names')
args = parser.parse_args(args=[])

---
# Getting of the predictions and labels

**Possible values:**

>Model directory *args.test_model_dir:* 

                                        'models/MobileNet_V2'
                                        'models/MobileNet_V2_ft'
                                        'models/ResNet_V2'
                                        'models/ResNet_V2_ft'
                                        'models/Inception_V3'
                                        'models/Inception_V3_ft' 

>Testing files *args.test_files:*

                                        ['data/img/001_001.png', 'data/img/013_009.png',..........]

>Label threshold *thresh_label:*

                                        0.00
                                        ...
                                        1.00

>Coordinate thresholds *thresh_x* and *thresh_y:*

                                                        0.00
                                                        ...
                                                        1.00

>Verbosity mode *verbose*:

                                        0 (quiet, no visualization)
                                        1 (visualization)

>Point shape *shape*:

                                        'star'
                                        'square'
                                        'circle'
                                
>Show point label *add_label*:

                                        True
                                        False

>Show point probability *add_prob*:

                                        True
                                        False    

>Color palette 'palette':

                                        'pastel'
                                        'hls'
                                        'Paired'
                                        'Set2'
                                        'Set3'  

---
> **Tested model:** MobileNet_V2
>
> **Visualize predictions:** No
>
> **Save images:** No

In [106]:
args.test_model_dir = 'models/MobileNet_V2'
args.test_files = ['data/img/017_011.png', 'data/img/013_023.png']
model_output = test_model(test_model_dir=args.test_model_dir, test_files=args.test_files)
images, labels, probs, coords = process_predictions(model_output=model_output, test_files=args.test_files, thresh_label=0.50, thresh_x=0.01, thresh_y=0.01)

----------------------------------------------------------------------------------------------------
Testing options:
Test model name...: MobileNet_V2
Test model dir....: models/MobileNet_V2
Tested images.....: ['privet.png']
----------------------------------------------------------------------------------------------------
----------------------------------------------------------------------------------------------------
Loading the model and its weights...
Loading the model and its weights takes (2.130 seconds)
----------------------------------------------------------------------------------------------------
----------------------------------------------------------------------------------------------------
Total pre-processing time......: 0.271 seconds
Average pre-processing time....: 0.271 seconds per image
Average pre-processing FPS.....: 3.7 frames per second
----------------------------------------------------------------------------------------------------
Total prediction 

In [109]:
show_predictions(images, labels, probs, coords, verbose=0, shape='star', add_label=False, add_prob=False, palette='pastel')

----------------------------------------------------------------------------------------------------
Image...........: privet.png
Labels..........: ['CD', 'CM', 'CT', 'PT']
Probabilities...: ['CD', 'CM', 'CT', 'PT']
X coordinates...: [112, 103, 189, 215]
Y coordinates...: [193, 167, 322, 301]
----------------------------------------------------------------------------------------------------


---
> **Tested model:** ResNet V2
>
> **Visualize predictions:** Yes
>
> **Save images:** No

In [0]:
args.test_model_dir = 'models/ResNet_V2'
args.test_files = ['data/img/017_011.png', 'data/img/013_023.png']
model_output = test_model(test_model_dir=args.test_model_dir, test_files=args.test_files)
images, labels, probs, coords = process_predictions(model_output=model_output, test_files=args.test_files, thresh_label=0.50, thresh_x=0.01, thresh_y=0.01)

In [0]:
show_predictions(images, labels, probs, coords, verbose=1, shape='star', add_label=True, add_prob=True, palette='pastel')

---
> **Tested model:** ResNet V2 (fine-tuned)
>
> **Visualize predictions:** No
>
> **Save images:** Yes

In [0]:
args.test_model_dir = 'models/ResNet_V2'
args.test_files = ['data/img/017_011.png', 'data/img/013_023.png']
model_output = test_model(test_model_dir=args.test_model_dir, test_files=args.test_files)
images, labels, probs, coords = process_predictions(model_output=model_output, test_files=args.test_files, thresh_label=0.50, thresh_x=0.01, thresh_y=0.01)

In [85]:
save_predictions(images, labels, probs, coords, save_dir='predictions', shape='star', add_label=True, add_prob=True, palette='pastel')

Image 017_011.png saved to predictions/MobileNet_V2/017_011_MobileNet_V2.png
Image 013_023.png saved to predictions/MobileNet_V2/013_023_MobileNet_V2.png
